# Gestion du Risque - Samsung Electronics
# Risk Management - Samsung Electronics

Ce notebook Jupyter implémente des modèles de calcul stochastique pour la gestion du risque des investissements en actions Samsung Electronics.

This Jupyter notebook implements stochastic calculus models for risk management of Samsung Electronics stock investments.

## Contenu / Contents
1. Configuration et imports
2. Modèle Black-Scholes pour le pricing d'options
3. Simulations Monte Carlo
4. Analyse de Risque (VaR, CVaR)
5. Gestion de Portfolio
6. Visualisations interactives

## 1. Configuration et Imports / Setup and Imports

In [ ]:
# Installation des dépendances (si nécessaire)
# Uncomment if you need to install dependencies
# !pip install numpy scipy matplotlib pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Configuration pour des graphiques de meilleure qualité
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Modules importés avec succès / Modules imported successfully")

## 2. Paramètres Samsung Electronics / Samsung Electronics Parameters

In [ ]:
# Paramètres pour Samsung Electronics
SAMSUNG_PARAMS = {
    'prix_actuel': 71000,        # Prix actuel en KRW / Current price in KRW
    'volatilite': 0.30,          # Volatilité annuelle 30% / Annual volatility 30%
    'rendement_attendu': 0.10,   # Rendement attendu 10% / Expected return 10%
    'taux_sans_risque': 0.035,   # Taux sans risque 3.5% / Risk-free rate 3.5%
    'montant_investissement': 10_000_000  # 10M KRW invested
}

# Affichage des paramètres
df_params = pd.DataFrame.from_dict(SAMSUNG_PARAMS, orient='index', columns=['Valeur / Value'])
print("Paramètres Samsung Electronics:")
print(df_params)

## 3. Modèle Black-Scholes / Black-Scholes Model

In [ ]:
class BlackScholesModel:
    """Modèle Black-Scholes pour le pricing d'options"""
    
    def __init__(self, S0, K, T, r, sigma):
        self.S0 = S0      # Prix actuel
        self.K = K        # Prix d'exercice
        self.T = T        # Temps jusqu'à échéance
        self.r = r        # Taux sans risque
        self.sigma = sigma # Volatilité
    
    def d1(self):
        return (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def call_price(self):
        """Prix d'une option d'achat"""
        return self.S0 * norm.cdf(self.d1()) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2())
    
    def put_price(self):
        """Prix d'une option de vente"""
        return self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2()) - self.S0 * norm.cdf(-self.d1())
    
    def delta_call(self):
        return norm.cdf(self.d1())
    
    def delta_put(self):
        return -norm.cdf(-self.d1())

print("✓ Classe BlackScholesModel définie / BlackScholesModel class defined")

### Exemple d'analyse d'options / Options Analysis Example

In [ ]:
# Analyser des options pour différents prix d'exercice
strike_prices = np.arange(65000, 80000, 2500)
days_to_expiry = 90
T = days_to_expiry / 365

results = []
for K in strike_prices:
    bs = BlackScholesModel(
        S0=SAMSUNG_PARAMS['prix_actuel'],
        K=K,
        T=T,
        r=SAMSUNG_PARAMS['taux_sans_risque'],
        sigma=SAMSUNG_PARAMS['volatilite']
    )
    results.append({
        'Strike': K,
        'Call Price': bs.call_price(),
        'Put Price': bs.put_price(),
        'Call Delta': bs.delta_call(),
        'Put Delta': bs.delta_put()
    })

df_options = pd.DataFrame(results)
print(f"\nAnalyse d'Options Samsung - {days_to_expiry} jours jusqu'à échéance:")
print(df_options.to_string(index=False))

In [ ]:
# Visualisation des prix d'options
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Prix des options
ax1.plot(df_options['Strike'], df_options['Call Price'], 'b-', label='Call', linewidth=2)
ax1.plot(df_options['Strike'], df_options['Put Price'], 'r-', label='Put', linewidth=2)
ax1.axvline(x=SAMSUNG_PARAMS['prix_actuel'], color='green', linestyle='--', label='Prix Actuel')
ax1.set_xlabel('Prix d\'Exercice (KRW)')
ax1.set_ylabel('Prix de l\'Option (KRW)')
ax1.set_title('Prix des Options en fonction du Strike')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Delta des options
ax2.plot(df_options['Strike'], df_options['Call Delta'], 'b-', label='Call Delta', linewidth=2)
ax2.plot(df_options['Strike'], df_options['Put Delta'], 'r-', label='Put Delta', linewidth=2)
ax2.axvline(x=SAMSUNG_PARAMS['prix_actuel'], color='green', linestyle='--', label='Prix Actuel')
ax2.set_xlabel('Prix d\'Exercice (KRW)')
ax2.set_ylabel('Delta')
ax2.set_title('Delta des Options')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Simulations Monte Carlo / Monte Carlo Simulations

In [ ]:
def simulate_stock_paths(S0, mu, sigma, T, dt, n_simulations):
    """
    Simule les trajectoires de prix selon un mouvement brownien géométrique
    Simulate price paths using geometric Brownian motion
    """
    n_steps = int(T / dt)
    paths = np.zeros((n_simulations, n_steps + 1))
    paths[:, 0] = S0
    
    for t in range(1, n_steps + 1):
        Z = np.random.standard_normal(n_simulations)
        paths[:, t] = paths[:, t-1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
        )
    
    return paths

print("✓ Fonction de simulation définie / Simulation function defined")

In [ ]:
# Simulation sur 1 an (252 jours de trading)
np.random.seed(42)  # Pour la reproductibilité

paths = simulate_stock_paths(
    S0=SAMSUNG_PARAMS['prix_actuel'],
    mu=SAMSUNG_PARAMS['rendement_attendu'],
    sigma=SAMSUNG_PARAMS['volatilite'],
    T=1.0,
    dt=1/252,
    n_simulations=10000
)

print(f"Simulations Monte Carlo complétées: {paths.shape[0]} trajectoires sur {paths.shape[1]} jours")

In [ ]:
# Visualisation des simulations
plt.figure(figsize=(14, 6))

# Tracer un sous-ensemble de trajectoires
for i in range(100):
    plt.plot(paths[i], alpha=0.3, linewidth=0.5, color='blue')

# Trajectoire moyenne
mean_path = np.mean(paths, axis=0)
plt.plot(mean_path, color='red', linewidth=3, label='Trajectoire Moyenne')

# Percentiles
p5 = np.percentile(paths, 5, axis=0)
p95 = np.percentile(paths, 95, axis=0)
plt.plot(p5, color='orange', linewidth=2, linestyle='--', label='Percentile 5%')
plt.plot(p95, color='orange', linewidth=2, linestyle='--', label='Percentile 95%')

plt.axhline(y=SAMSUNG_PARAMS['prix_actuel'], color='black', linestyle='--', 
           label=f'Prix Initial: {SAMSUNG_PARAMS["prix_actuel"]:,.0f} KRW')
plt.xlabel('Jours de Trading')
plt.ylabel('Prix de l\'Action (KRW)')
plt.title('Simulation Monte Carlo - Samsung Electronics (10,000 trajectoires)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Analyse de Risque / Risk Analysis

In [ ]:
# Calcul des métriques de risque
final_prices = paths[:, -1]
returns = (final_prices - SAMSUNG_PARAMS['prix_actuel']) / SAMSUNG_PARAMS['prix_actuel']
portfolio_values = SAMSUNG_PARAMS['montant_investissement'] * (1 + returns)
portfolio_losses = SAMSUNG_PARAMS['montant_investissement'] - portfolio_values

# Statistiques de base
stats = {
    'Prix Moyen Final': np.mean(final_prices),
    'Prix Médian Final': np.median(final_prices),
    'Écart-type Prix': np.std(final_prices),
    'Rendement Moyen': np.mean(returns),
    'Rendement Médian': np.median(returns),
    'Volatilité Réalisée': np.std(returns),
    'Probabilité de Perte': np.sum(returns < 0) / len(returns)
}

df_stats = pd.DataFrame.from_dict(stats, orient='index', columns=['Valeur'])
print("\nStatistiques de Risque - Samsung Electronics:")
print("=" * 60)
for key, value in stats.items():
    if 'Prix' in key or 'Écart' in key:
        print(f"{key:30s}: {value:12,.2f} KRW")
    elif 'Probabilité' in key:
        print(f"{key:30s}: {value:12.2%}")
    else:
        print(f"{key:30s}: {value:12.4f}")

### Value at Risk (VaR) et Conditional VaR (CVaR)

In [ ]:
# Calcul de la VaR à différents niveaux de confiance
confidence_levels = [0.90, 0.95, 0.99]

print("\nValue at Risk (VaR) Analysis:")
print("=" * 60)

var_results = []
for conf in confidence_levels:
    # VaR en rendement
    var_return = np.percentile(returns, (1 - conf) * 100)
    # VaR en valeur monétaire
    var_monetary = np.percentile(portfolio_losses, conf * 100)
    
    # CVaR (Expected Shortfall)
    cvar_return = np.mean(returns[returns <= var_return])
    
    var_results.append({
        'Niveau de Confiance': f"{conf:.0%}",
        'VaR (Rendement)': f"{var_return:.2%}",
        'VaR (Perte Max en KRW)': f"{var_monetary:,.0f}",
        'CVaR (Rendement)': f"{cvar_return:.2%}"
    })
    
    print(f"\nConfiance {conf:.0%}:")
    print(f"  VaR (Rendement):          {var_return:+.2%}")
    print(f"  VaR (Perte Max):          {var_monetary:,.0f} KRW")
    print(f"  CVaR (Expected Shortfall): {cvar_return:+.2%}")

df_var = pd.DataFrame(var_results)
print("\n" + "=" * 60)

In [ ]:
# Visualisation de la distribution des rendements
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme des rendements
ax1.hist(returns, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax1.axvline(x=0, color='black', linestyle='--', label='Rendement Nul')
ax1.axvline(x=np.percentile(returns, 5), color='red', linestyle='--', 
           label=f'VaR 95%: {np.percentile(returns, 5):.2%}')
ax1.axvline(x=np.mean(returns), color='green', linestyle='--', 
           label=f'Rendement Moyen: {np.mean(returns):.2%}')
ax1.set_xlabel('Rendement')
ax1.set_ylabel('Fréquence')
ax1.set_title('Distribution des Rendements')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Histogramme des pertes du portfolio
losses_only = portfolio_losses[portfolio_losses > 0]
ax2.hist(losses_only, bins=50, alpha=0.7, color='red', edgecolor='black')
ax2.axvline(x=np.percentile(portfolio_losses, 95), color='darkred', linestyle='--', 
           label=f'VaR 95%: {np.percentile(portfolio_losses, 95):,.0f} KRW')
ax2.set_xlabel('Perte du Portfolio (KRW)')
ax2.set_ylabel('Fréquence')
ax2.set_title('Distribution des Pertes')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Stratégies de Gestion du Risque / Risk Management Strategies

### Stratégie de Hedging avec Options / Hedging Strategy with Options

In [ ]:
# Évaluer une stratégie de protection avec des puts
# Evaluate a protective put strategy

# Acheter un put à 90% du prix actuel (protective put)
protection_strike = SAMSUNG_PARAMS['prix_actuel'] * 0.90
protection_expiry_days = 180  # 6 mois

bs_protection = BlackScholesModel(
    S0=SAMSUNG_PARAMS['prix_actuel'],
    K=protection_strike,
    T=protection_expiry_days / 365,
    r=SAMSUNG_PARAMS['taux_sans_risque'],
    sigma=SAMSUNG_PARAMS['volatilite']
)

put_cost = bs_protection.put_price()
num_shares = SAMSUNG_PARAMS['montant_investissement'] / SAMSUNG_PARAMS['prix_actuel']
total_protection_cost = put_cost * num_shares

print("Stratégie de Protection avec Put Options:")
print("=" * 60)
print(f"Nombre d'actions détenues:        {num_shares:,.0f}")
print(f"Prix d'exercice de protection:    {protection_strike:,.0f} KRW")
print(f"Coût du put par action:           {put_cost:,.2f} KRW")
print(f"Coût total de protection:         {total_protection_cost:,.0f} KRW")
print(f"Coût en % de l'investissement:    {(total_protection_cost/SAMSUNG_PARAMS['montant_investissement']):.2%}")
print(f"\nPerte maximale protégée:          {(SAMSUNG_PARAMS['prix_actuel'] - protection_strike) * num_shares:,.0f} KRW")
print(f"Perte max en %:                   {((protection_strike - SAMSUNG_PARAMS['prix_actuel']) / SAMSUNG_PARAMS['prix_actuel']):.2%}")

### Comparaison: Portfolio avec et sans Protection / Comparison: Portfolio with and without Protection

In [ ]:
# Simuler les rendements avec protection
protected_values = np.maximum(portfolio_values - total_protection_cost, 
                               protection_strike * num_shares - total_protection_cost)

# Calculer les pertes pour les deux stratégies
unprotected_losses = SAMSUNG_PARAMS['montant_investissement'] - portfolio_values
protected_losses = SAMSUNG_PARAMS['montant_investissement'] - protected_values

comparison = {
    'Métrique': [
        'Valeur Moyenne Finale',
        'Perte Moyenne',
        'Perte Maximale',
        'VaR 95%',
        'CVaR 95%'
    ],
    'Sans Protection': [
        np.mean(portfolio_values),
        np.mean(unprotected_losses[unprotected_losses > 0]),
        np.max(unprotected_losses),
        np.percentile(unprotected_losses, 95),
        np.mean(unprotected_losses[unprotected_losses >= np.percentile(unprotected_losses, 95)])
    ],
    'Avec Protection': [
        np.mean(protected_values),
        np.mean(protected_losses[protected_losses > 0]),
        np.max(protected_losses),
        np.percentile(protected_losses, 95),
        np.mean(protected_losses[protected_losses >= np.percentile(protected_losses, 95)])
    ]
}

df_comparison = pd.DataFrame(comparison)
df_comparison['Sans Protection'] = df_comparison['Sans Protection'].apply(lambda x: f"{x:,.0f} KRW")
df_comparison['Avec Protection'] = df_comparison['Avec Protection'].apply(lambda x: f"{x:,.0f} KRW")

print("\nComparaison des Stratégies de Gestion du Risque:")
print("=" * 80)
print(df_comparison.to_string(index=False))

In [ ]:
# Visualisation de la comparaison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des valeurs finales
ax1.hist(portfolio_values/1e6, bins=50, alpha=0.5, label='Sans Protection', color='blue')
ax1.hist(protected_values/1e6, bins=50, alpha=0.5, label='Avec Protection', color='green')
ax1.axvline(x=SAMSUNG_PARAMS['montant_investissement']/1e6, color='red', linestyle='--', 
           label='Investissement Initial')
ax1.set_xlabel('Valeur du Portfolio (Millions KRW)')
ax1.set_ylabel('Fréquence')
ax1.set_title('Distribution des Valeurs Finales du Portfolio')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Comparaison des pertes
ax2.hist(unprotected_losses/1e6, bins=50, alpha=0.5, label='Sans Protection', color='red', range=(0, 8))
ax2.hist(protected_losses/1e6, bins=50, alpha=0.5, label='Avec Protection', color='orange', range=(0, 8))
ax2.set_xlabel('Perte (Millions KRW)')
ax2.set_ylabel('Fréquence')
ax2.set_title('Distribution des Pertes')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Résumé et Recommandations / Summary and Recommendations

In [ ]:
print("═" * 80)
print("RÉSUMÉ DE L'ANALYSE DE RISQUE - SAMSUNG ELECTRONICS")
print("RISK ANALYSIS SUMMARY - SAMSUNG ELECTRONICS")
print("═" * 80)
print()
print("PARAMÈTRES D'INVESTISSEMENT / INVESTMENT PARAMETERS:")
print(f"  Investissement initial:           {SAMSUNG_PARAMS['montant_investissement']:>15,.0f} KRW")
print(f"  Prix actuel Samsung:              {SAMSUNG_PARAMS['prix_actuel']:>15,.0f} KRW")
print(f"  Nombre d'actions:                 {num_shares:>15,.0f}")
print()
print("RÉSULTATS DES SIMULATIONS (10,000 scénarios) / SIMULATION RESULTS:")
print(f"  Prix moyen final:                 {np.mean(final_prices):>15,.0f} KRW")
print(f"  Rendement attendu:                {np.mean(returns):>15.2%}")
print(f"  Probabilité de perte:             {np.sum(returns < 0) / len(returns):>15.2%}")
print()
print("MÉTRIQUES DE RISQUE / RISK METRICS:")
print(f"  VaR 95% (rendement):              {np.percentile(returns, 5):>15.2%}")
print(f"  VaR 95% (perte en KRW):           {np.percentile(portfolio_losses, 95):>15,.0f} KRW")
print(f"  CVaR 95% (rendement):             {np.mean(returns[returns <= np.percentile(returns, 5)]):>15.2%}")
print()
print("STRATÉGIE DE PROTECTION / PROTECTION STRATEGY:")
print(f"  Coût de protection (put options): {total_protection_cost:>15,.0f} KRW")
print(f"  Perte maximale avec protection:   {np.max(protected_losses):>15,.0f} KRW")
print(f"  Réduction de la VaR 95%:          {np.percentile(unprotected_losses, 95) - np.percentile(protected_losses, 95):>15,.0f} KRW")
print()
print("RECOMMANDATIONS / RECOMMENDATIONS:")
print("  1. Diversification du portfolio recommandée")
print("  2. Considérer une stratégie de protection si l'aversion au risque est élevée")
print("  3. Surveillance régulière de la volatilité du marché")
print("  4. Réévaluation trimestrielle des positions")
print()
print("⚠️  AVERTISSEMENT: Cette analyse est à des fins éducatives uniquement.")
print("    Consultez un conseiller financier professionnel avant d'investir.")
print("═" * 80)

## Exercices Supplémentaires / Additional Exercises

Pour approfondir votre compréhension, essayez ces exercices:

1. **Modifier les paramètres**: Changez la volatilité, le rendement attendu et observez l'impact sur la VaR
2. **Analyse de sensibilité**: Créez des graphiques montrant comment la VaR change avec différents niveaux de volatilité
3. **Différentes stratégies d'options**: Implémentez un collar (achat de put + vente de call)
4. **Backtesting**: Utilisez des données historiques réelles de Samsung pour valider les modèles
5. **Portfolio multi-actifs**: Ajoutez d'autres actions coréennes et analysez la diversification

---

**Références / References:**
- Black, F., & Scholes, M. (1973). The Pricing of Options and Corporate Liabilities
- Hull, J. C. (2018). Options, Futures, and Other Derivatives
- Jorion, P. (2006). Value at Risk: The New Benchmark for Managing Financial Risk